### Imports

In [1]:
import requests

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

import numpy as np
import pandas as pd

### Connecting to ArcGIS

In [2]:
# Login for the notebook running in AGOL
#gis = GIS("home")

# Login for the notebook running in Pro locally...
gis = GIS("pro")

### Variables
Currently the production layer (which is already powering the very very pretty Dashboard) is commented out and I'm grabbing a test layer instead (next block down). Will un-comment the operational layer when I'm 100% sure things aren't blowing up anymore.

In [ ]:

# The Item ID of the service containing both the geometry layers and the dashboard layer
# dd is "Disaster Declarations"
dd_id = "d37c3c2a6f1c4586baad82828bfc3c59"

# Get the item at this item ID
dd_item = gis.content.get(dd_id)

# Item ID 1 is the input layer used for getting geometries
geometries_layer = dd_item.layers[1]

print(geometries_layer)

# Item ID 0 is the output layer displayed in the dashboard
# COMMENTING OUT WHILE TESTING
#dashboard_layer = dd_item.layers[0]

#print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/Disaster_Declarations_Summaries_v2/FeatureServer/1">


### Get the Dashboard layer FOR TESTING ONLY

In [20]:
# ID of the item including Disaster Declarations Summaries subset FOR TESTING ONLY,
# replaces "dashboard_layer" above through duration of testing
test_id = "edb716da51bc4f7882d13d425ad08fd2"

test_item = gis.content.get(test_id)

dashboard_layer = test_item.layers[0]

print(dashboard_layer)

<FeatureLayer url:"https://services9.arcgis.com/GDVaV4SDJDDBT8gi/arcgis/rest/services/DisasterDeclarations_forTesting_2025only/FeatureServer/0">


### Build dict of primary/foreign keys
I'm thinking I'll need a little dict of the various fields I'll need to comparefor querying the geometries layer...? The key is the field name in the FEMA data; the value is the corresponding field in my geometries layer... ACTUALLY I need to do that backwards because placeCODE would create duplicate keys... SO the key will be in the geometries; the value will be the FEMA data

In [5]:
# Just updated the values below to reflect the fields I've added to the Summaries df; they should be GTG now
key_dict = {
    "State_FIPS": "fipsStateCode",
    "Full_FIPS": "fipsFullCode",
    "AIANNHFP1": "fipsTribalCode",
    "AIANNHFP2": "fipsTribalCode", # No idea why there are three of these...or why the Summaries sometimes use 2 and 3...
    "AIANNHFP3": "fipsTribalCode" # What a pain in the @$$...
}

I guess the above block combined with the below block might be a good way to reference the correct fields; it will either just be state_field or key_dict[state_field] to point to either the geometries layer field or the dataframe column, respectively. We'll see if this remains relevant as the script developes.

In [6]:
# I believe I will need three different queries for my geometries layer
# I'll need to query to get state geometries, county geometries and tribal area geometries:

# Get the states on this field:
# (States we'll check first; )
state_field = "State_FIPS"

# Get the counties on this field:
county_field = "Full_FIPS"

# Crappily, in assembling the original dashboard layer I realized that
# the Declaration data may match ANY ONE of these tribal fields, so I need to check all three:
tribal_field1 = "AIANNHFP1"
tribal_field2 = "AIANNHFP2"
tribal_field3 = "AIANNHFP3"

### Connect to OpenFEMA API and get Disaster Declarations Summaries
* Right right right; I forgot that the API by default only returns 1000 records. I shouldn't really NEED more records than that, since the script is going to be run once per day. One thousand records should be MORE than enough. But, I now realize I need to do a little footwork to make sure I am just getting the 1000 _most recent_ records...

* Okay added a sort order to the api call to only get the most recent records by declarationDate! That should do it...

In [7]:
# Within the API URL, filter the records to return only fyDeclared to 2013 or newer.
# US Census GDBs only go back to 2013; before that it's shapefiles only
# and I refuse to touch shapefiles, at least for the scope of this project.

api_url = r"https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$filter=fyDeclared ge 2013&$orderby=declarationDate desc"

# Plug in the URL and capture the response obj
response = requests.get(api_url)

# Convert response to JSON
data = response.json()

# Okay so after a little digging I really only need the following (leave out the metadata)
summaries_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])

summaries_df.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5612-CA,5612,CA,FM,2025-09-03T00:00:00.000Z,2025,Fire,2-7 FIRE,False,False,...,99009,Calaveras (County),25121,None,2025090301,9,R,2025-09-03T18:41:07.857Z,d017531813b75fc753371c26b246931d48de651e,28a1ba9f-d914-4024-9e75-4a66b5bba092
1,FM-5611-MT,5611,MT,FM,2025-08-26T00:00:00.000Z,2025,Fire,WINDY ROCK FIRE,False,False,...,99077,Powell (County),25119,None,2025082701,8,R,2025-08-28T18:01:23.160Z,29e175a73b969da6864182e703e3cb3f8d0bb32d,41329e57-2046-4196-a63d-902f3e7c923c
2,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99017,Deschutes (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,c4a190d030807595da90813aabc6ad2175917668,df7cb24f-8e5a-4c1e-923e-4c75c9ec4581
3,FM-5610-OR,5610,OR,FM,2025-08-23T00:00:00.000Z,2025,Fire,FLAT FIRE,False,False,...,99031,Jefferson (County),25117,None,2025082301,10,R,2025-08-25T18:21:58.453Z,8b07b29243bdbba511790332bd3fa9cca0fe33fd,f0604c05-113b-449e-8e4a-f3b5076af546
4,FM-5609-HI,5609,HI,FM,2025-08-19T00:00:00.000Z,2025,Fire,KUNIA ROAD FIRE,False,False,...,99003,Honolulu (County),25114,None,2025082001,9,R,2025-08-21T18:22:16.374Z,731df26a647e5a0338177f445bab7a23b6f8d6ed,ffab7fa0-2e69-428d-b4d3-da95ca352c03


### Coffee Break 9/3
Okay yeah it's a mess in here right now. Early development stages. Pardon the dust.
Need to figure out exactly how this whole thing is going to work...
...below begin the algorithms...

Things that will have to be done, in no particular order:

* BEFORE I dissolve (or the pd equivalent of dissolve) the summaries to the FEMA Declaration String level, I need to get all the counties / tribal areas associated with that string so I can grab their geometries and actually perform the spatial dissolve on them
* Tricky with the above: If the FEMA Declaration String applies to both counties and tribal areas, these are treated separately
* Once I have the list of entities the FEMA Declaration String is for, grab the geometries for those counties and perform the dissolve on them. That new dissolve geometry will be the geometry applied to the new row written to the output Summaries dataset.
* THEN dissolve (pd equivalent) the actual summaries, ensuring the schema matches that of the target Summaries dataset...tack on the geometry, apply edits.
* Other tricky bits: Obviously we need to check if a given FEMA string already exists within the summaries dataset. I will need to check whether this is as straightforward as it sounds, or whether...hm. I will have to check whether the potential exists for Summary rows with the same FEMA String to be issued across multiple days...e.g., this string for these three counties is issued this day; then another three rows for an additional three counties are added UNDER THE SAME STRING...this will complicate the checking process.
* Yet more tricky bits: The fields from the summaries used to match the data to the appropriate geometries is different depending on the level of the entity being matched.

For example, Statewide declarations will match on state fips, of course. Counties will match on a combination of state + county code (I could calc that field in the df before I begin...?), and tribal entities will match on...wtf will they match on again...I believe it's a concat of the state FIPS and placeCode...yes because the long tribal codes in the geometries are seven chars...I think. 🥴

I could have just gone easy on myself and committed to representing only county-based declaration rows in my map. But does that simplicity accurately reflect the real world? NO. Does it create a more impressive script? NO! Does it get me a job faster? NO!! 😤

### Convert pseudo-date columns to actual date columns
I checked all the fips / code fields to ensure they're object / string type (I deleted that block while tidying up), so the offending fields remaining are the pseudo-date fields

In [8]:
date_columns = ["declarationDate", "incidentBeginDate", "incidentEndDate", "disasterCloseoutDate", "lastIAFilingDate", "lastRefresh"]

for dc in date_columns:
    summaries_df[dc] = pd.to_datetime(summaries_df[dc], errors="coerce")

### Add full FIPS Code (counties) and full Tribal Code columns

In [9]:
# Add to my df the fields I will need for comparison
summaries_df["fipsFullCode"] = summaries_df["fipsStateCode"] + summaries_df["fipsCountyCode"]
summaries_df["fipsTribalCode"] = summaries_df["fipsStateCode"] + summaries_df["placeCode"]

print(summaries_df["fipsFullCode"].head())
print(summaries_df["fipsTribalCode"].head())

0    06009
1    30077
2    41017
3    41031
4    15003
Name: fipsFullCode, dtype: object
0    0699009
1    3099077
2    4199017
3    4199031
4    1599003
Name: fipsTribalCode, dtype: object


* Considering how I'm going to get the data from the API in shape for the comparison etc. I should just add the two additional columns I added manually for the dashboard I made first, COVID and Entity. After I add and calculate them, the comparisons will all be much easier, because I can just reference those fields for processing the data in chunks (i.e. step 1 process statewide, step 2 process counties, step 3 process tribal)

### Add & calc "COVID19" field

In [10]:
# Good lord I can't remember how to calculate any of these fields with pandas... 🤣😭
# Anyway the first one I need to calc is the COVID column, simple yes/no

summaries_df["COVID19"] = np.where(summaries_df["declarationTitle"].str.contains("COVID-19"), "Show only COVID-19", "Show only non-COVID-19")


### Add & calc "Entities" field

In [11]:
# For my next trick I'll use np.select instead of np.where since to code new Entity column
# I have three possible values not just 2 / yes no / on off

entity_conditions = [
    summaries_df["designatedArea"] == "Statewide",
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] == "000"),
    (summaries_df["designatedArea"] != "Statewide") & (summaries_df["fipsCountyCode"] != "000")
]

entity_values = ["State or Equivalent", "Tribal Area or Equivalent", "County or Equivalent"]

summaries_df["Entity"] = np.select(entity_conditions, entity_values)

### Coffee Break 9/4
Nice! Okay! So far, we've added the full FIPS code (5 digit) and the full tribal code (7-digit). I've also added the COVID field (basically yes or no) and the Entity field (State, County or Tribal). I mean that's good progress for feeling crappy the last few days (not like, sick crappy, just Michigan-is-so-f*cking-lame crappy).

Now, what needs to be done next?

* I first need to compare the values that are already in the dissolved Summaries layer (the dashboard layer) with what's in my dataframe.
* Before I do that, I need to figure out exactly what field we're going to compare on. My first impulse is to use the FEMA Declaration String, but as we discussed above, not sure if that's going to work well. For example...if FEMA pushes out twenty counties with one FEMA Dec String one day, then they decide to push out another ten counties with the same Dec String the next day, if I do the comparison the way I intend, I'll fine that Dec String in my dashboard already and ignore it. I won't get those ten new counties.
* I guess I really need to root around in the data to see if my impulse is a valid way to compare the data...

Okay so I'll do that in a minute here. What needs to happen then?
* I'll need to do the actual comparison, whatever that looks like. Hopefully it's just looking for FEMA Dec Strings in the Dashboard and it's as simple as that.
* I'd probably just keep a running tally of the Dec Strings that are already in the dashboard and then just drop those records from my df (technically make a new df where I've dropped those records)
* Once I've isolated the new Dec Strings, I can't dissolve them yet; I need to get the geometries of the associated areas.
* I get all the geometries associated with a given Dec String, do the dissolve thingy (the actual dissolve gp tool thingy), then...well that's as far as my brain needs to go right now.

First let's analyze the temporal spans of all observations with some help with your friend and mine ChatGPT...

### Groupby "femaDeclarationString", get temporal range (prelim analysis; to delete)

In [12]:
analyze = summaries_df.groupby("femaDeclarationString")["declarationDate"].agg(["min", "max"])

analyze["range"] = (analyze["max"] - analyze["min"]).dt.total_seconds() / 3600

analyze["range"].max()

0.0

Well that's encouraging (no really I'm not even being sarcastic). I get a whole column of big fat zeros for the declaredDate ranges of all the fema Dec Strings. Honestly...that means I should be fine just proceeding with my simple it's-there-or-not check. I should probably also commit.

OK getting near end-of-day. But now I know I can go forward with a comparison so maybe I'll start that part...

### Get unique femaDeclarationStrings from FEMA API

In [22]:
new_fema_strings = summaries_df["femaDeclarationString"].unique().tolist()

### Get unique femaDeclarationString vals from Dashboard

In [ ]:
dashboard_sdf = dashboard_layer.query().sdf

existing_fema_strings = dashboard_sdf["femaDeclarationString"].unique().tolist()

100

### Get only the fema strings we need to worry about

In [26]:
adds_strings = [s for s in new_fema_strings if not s in existing_fema_strings]

### Create a new dataframe with only the fema strings we want

In [27]:
adds_df = summaries_df[summaries_df["femaDeclarationString"].isin(keep_fema_strings)]

Check back tomorrow for another exciting episode! Hopefully we can grab & dissolve the geometries for the keep-rows from our fema dataframe.